# 初始化設定

## 將程式下載到colab環境

In [1]:
!git clone --single-branch https://github.com/zengxianyu/crfill

Cloning into 'crfill'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 304 (delta 50), reused 44 (delta 44), pack-reused 247 (from 1)
Receiving objects: 100% (304/304), 19.78 MiB | 7.21 MiB/s, done.
Resolving deltas: 100% (106/106), done.


## 設定環境與參數

In [2]:
%cd crfill

/content/crfill


In [3]:
#!conda env create -f environment.yml

#!pip install torch==1.3.1 torchvision
#!pip install opencv-python==3.4.2
!pip install tqdm dill matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.9 MB/s eta 0:00:00


In [4]:
!git submodule init
!git submodule update

Submodule 'logger' (https://github.com/zengxianyu/logger) registered for path 'logger'
Submodule 'models/networks/sync_batchnorm' (https://github.com/zengxianyu/sync_batchnorm) registered for path 'models/networks/sync_batchnorm'
Cloning into '/content/crfill/logger'...
Cloning into '/content/crfill/models/networks/sync_batchnorm'...
Submodule path 'logger': checked out '70b189abb9a894b6089bf51c5d29b480ed5529c8'
Submodule path 'models/networks/sync_batchnorm': checked out 'da7372f8f3acf4ce6401fb4a76b38ba1100b8663'


## 上傳資料集

In [5]:
#將資料集複製到執行目錄下(由於dataset.py的載入方法基於執行目錄的相對路徑，因此在不更動程式的情況下這樣處理)
import shutil

In [6]:
# 複製data資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/data"  # 請替換成你的data資料集的絕對路徑
target_path = "/content/crfill/mydata"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")


資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/data 複製到 /content/crfill/mydata


In [ ]:
#複製test資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/test"  # 請替換成你的test資料集的絕對路徑
target_path = "/content/crfill/mytest"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")

資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/test 複製到 /content/Pytorch-MADF/test


## 測試

In [5]:
!chmod +x download/*
!./download/download_model.sh
!./download/download_data.sh

串流輸出內容已截斷至最後 5000 行。
  inflating: datasets/object_masks/XPIE_masks/Y_I_620.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_5089.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_1891.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_2257.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_2205.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_5051.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_1731.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_8274.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_7064.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_4326.png  
  inflating: datasets/object_masks/XPIE_masks/Y_E_397.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_7073.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_6715.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_2121.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_6411.png  
  inflating: datasets/object_masks/XPIE_masks/Y_I_5998.png  
  inf

In [6]:
!./test.sh

----------------- Options ---------------
             aspect_ratio: 1.0                           
                batchSize: 1                             
      cache_filelist_read: False                         
     cache_filelist_write: False                         
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                d_mask_in: False                         
                 dataroot: ./datasets/cityscapes/        
             dataset_mode: testimage                     	[default: coco]
          display_winsize: 256                           
                  gpu_ids: 0                             
                 how_many: inf                           
                image_dir: ./datasets/places2sample1k_val/places2samples1k_crop256	[default: None]
                init_type: xavier                        
            init_variance: 0.02                          
                  isTrain: Fals

In [7]:
!./train.sh

----------------- Options ---------------
            D_steps_per_G: 1                             
             aspect_ratio: 1.0                           
                batchSize: 48                            	[default: 1]
                    beta1: 0.0                           
                    beta2: 0.9                           
      cache_filelist_read: False                         
     cache_filelist_write: False                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           	[default: 512]
                d_mask_in: False                         
                 dataroot: ./datasets/cityscapes/        
             dataset_mode: coco                          
       dataset_mode_train: trainimage                    	[default: coco]
         dataset_mode_val: valimage                      	[default: None]
                    debug: F

# Train place2

## 處理資料集

In [7]:
import os

def generate_image_list(folder_path):
    """
    Generate a .txt file containing all image filenames in a folder.

    Args:
        folder_path (str): Path to the folder containing images.
    """
    # Validate the folder path
    if not os.path.isdir(folder_path):
        raise ValueError(f"Provided path '{folder_path}' is not a valid directory.")

    # Get all image filenames in the folder
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
    image_files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in image_extensions]

    # Sort the filenames to ensure order
    image_files.sort()

    # Get the folder name and determine the output .txt path
    folder_name = os.path.basename(folder_path.rstrip(os.sep))
    parent_dir = os.path.dirname(folder_path)
    txt_path = os.path.join(parent_dir, f"{folder_name}.txt")

    # Write filenames to the .txt file
    with open(txt_path, 'w') as f:
        for image_file in image_files:
            f.write(image_file + '\n')

    print(f"Image list saved to: {txt_path}")

# Example usage:
# generate_image_list("/path/to/your/image_folder")


In [8]:
generate_image_list("/content/crfill/mydata/place2_train")
generate_image_list("/content/crfill/mydata/place2_val")
generate_image_list("/content/crfill/mydata/train_mask")
generate_image_list("/content/crfill/mydata/val_mask")

Image list saved to: /content/crfill/mydata/place2_train.txt
Image list saved to: /content/crfill/mydata/place2_val.txt
Image list saved to: /content/crfill/mydata/train_mask.txt
Image list saved to: /content/crfill/mydata/val_mask.txt


In [9]:
!pip install pycocotools


In [23]:
##BSIZE0=48 # stage coarse
BSIZE=96 # 96:64G
BSIZE0=BSIZE//2
NWK=16
#PREFIX="--dataset_mode_train trainimage --gpu_ids 0,1 --name debug --dataset_mode_val valimage --train_image_dir ./mydata/place2_train --train_image_list ./mydata/place2_train.txt --path_objectshape_list ./mydata/train_mask.txt --path_objectshape_base ./mydata/train_mask --val_image_dir ./mydata/place2_val --val_image_list ./mydata/place2_val.txt --val_mask_dir ./content/crfill/mydata/val_mask --no_vgg_loss --no_ganFeat_loss --load_size 640 --crop_size 256 --model inpaint --netG baseconv --netD deepfill --preprocess_mode scale_shortside_and_crop --validation_freq 10000 --gpu_ids 0,1 --niter 50 "
!python train.py --dataset_mode_train trainimage --gpu_ids 0 --name debug --dataset_mode_val valimage --train_image_dir ./mydata/place2_train --train_image_list ./mydata/place2_train.txt --path_objectshape_list ./mydata/train_mask.txt --path_objectshape_base ./mydata/train_mask --val_image_dir ./mydata/place2_val --val_image_list ./mydata/place2_val.txt --val_mask_dir ./content/crfill/mydata/val_mask --no_vgg_loss --no_ganFeat_loss --load_size 640 --crop_size 256 --model inpaint --netG baseconv --netD deepfill --preprocess_mode scale_shortside_and_crop --validation_freq 10000 --gpu_ids 0,1 --niter 50 --batchSize ${BSIZE0} --nThreads ${NWK} --no_fine_loss --update_part coarse --no_gan_loss --freeze_D --niter 1


----------------- Options ---------------
            D_steps_per_G: 1                             
             aspect_ratio: 1.0                           
                batchSize: 8                             	[default: 1]
                    beta1: 0.0                           
                    beta2: 0.9                           
      cache_filelist_read: False                         
     cache_filelist_write: False                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           	[default: 512]
                d_mask_in: False                         
                 dataroot: ./datasets/cityscapes/        
             dataset_mode: coco                          
       dataset_mode_train: trainimage                    	[default: coco]
         dataset_mode_val: valimage                      	[default: None]
                    debug: F

In [16]:
import torch
print(torch.cuda.device_count())  # 顯示可用 GPU 的數量


1


In [18]:
##BSIZE0=48 # stage coarse
BSIZE=96 # 96:64G
BSIZE0=BSIZE//2
NWK=16
PREFIX="--dataset_mode_train trainimage \
--gpu_ids 0,1 \
--name debug \
--dataset_mode_val valimage \
--train_image_dir ./mydata/place2_train \
--train_image_list ./mydata/place2_train.txt \
--path_objectshape_list ./mydata/train_mask.txt \
--path_objectshape_base ./mydata/train_mask \
--val_image_dir ./mydata/place2_val \
--val_image_list ./mydata/place2_val.txt \
--val_mask_dir ./content/crfill/mydata/val_mask \
--no_vgg_loss \
--no_ganFeat_loss \
--load_size 640 \
--crop_size 256 \
--model inpaint \
--netG baseconv \
--netD deepfill \
--preprocess_mode scale_shortside_and_crop \
--validation_freq 10000 \
--gpu_ids 0,1 \
--niter 50 "
!python train.py $PREFIX --batchSize $BSIZE0 --nThreads ${NWK} --no_fine_loss --update_part coarse --no_gan_loss --freeze_D --niter 1 ${EXTRA}
!python train.py \
	${PREFIX} \
	--batchSize ${BSIZE} \
	--nThreads ${NWK} \
	--update_part fine \
	--continue_train \
	--niter 2 \
	${EXTRA}
!python train.py \
	${PREFIX} \
	--batchSize ${BSIZE} \
	--nThreads ${NWK} \
	--update_part all \
	--continue_train \
	--niter 4 \
	${EXTRA}

PREFIX="--dataset_mode_train trainimage \
--name debugarr0 \
--gpu_ids 0,1 \
--dataset_mode_val valimage \
--train_image_dir ./mydata/place2_train \
--train_image_list ./mydata/place2_train.txt \
--path_objectshape_list ./mydata/train_mask.txt \
--path_objectshape_base ./mydata/train_mask \
--val_image_dir ./mydata/place2_val \
--val_image_list ./mydata/place2_val.txt \
--val_mask_dir ./content/crfill/mydata/val_mask \
--no_vgg_loss \
--no_ganFeat_loss \
--gpu_ids 0,1 \
--load_size 640 \
--crop_size 256 \
--model arrange \
--netG twostagend \
--baseG baseconv \
--norm_type 1 \
--netD deepfill \
--load_base_g ./checkpoints/debug/latest_net_G.pth \
--load_base_d ./checkpoints/debug/latest_net_D.pth \
--lambda_ref 0.5 \
--lambda_l1 1 \
--preprocess_mode scale_shortside_and_crop"
!python train.py \
	${PREFIX} \
	--batchSize ${BSIZE0} \
	--nThreads ${NWK} \
	--update_part all \
	--niter 10 \
	${EXTRA}


usage: train.py [-h] [--name NAME] [--gpu_ids GPU_IDS] [--checkpoints_dir CHECKPOINTS_DIR]
                [--model MODEL] [--phase PHASE] [--batchSize BATCHSIZE]
                [--preprocess_mode {resize_and_crop,crop,scale_width,scale_width_and_crop,scale_shortside,scale_shortside_and_crop,fixed,none}]
                [--load_size LOAD_SIZE] [--crop_size CROP_SIZE] [--aspect_ratio ASPECT_RATIO]
                [--output_nc OUTPUT_NC] [--dataroot DATAROOT] [--serial_batches] [--no_flip]
                [--nThreads NTHREADS] [--max_dataset_size MAX_DATASET_SIZE] [--load_from_opt_file]
                [--cache_filelist_write] [--cache_filelist_read]
                [--display_winsize DISPLAY_WINSIZE] [--netG NETG] [--init_type INIT_TYPE]
                [--init_variance INIT_VARIANCE] [--norm_type NORM_TYPE] [--use_th] [--th TH]
                [--trainer TRAINER] [--display_freq DISPLAY_FREQ] [--print_freq PRINT_FREQ]
                [--save_latest_freq SAVE_LATEST_FREQ] [--validation